<a href="https://colab.research.google.com/github/PeeratPookpanich/text_summarization_hw/blob/main/Text_sum_LSA_6410422017_Peerat_Pookpanich.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DADS7203 Natural Language Processing
##6410422017 Peerat Pookpanich

## Basic Text pre-processing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Get Text Document

In [ ]:
import base64
import requests

url = "https://raw.githubusercontent.com/PeeratPookpanich/text_summarization_hw/main/Transitioning%20from%20engineering%20to%20data%20science%20in%20Bangkok%20Thailand.txt"
article_content = requests.get(url)
article_content = article_content.text
print(article_content)

Data science is still the most attractive trend in Thailand nowadays. 
There are many open positions for data scientists, data engineers, and data-related roles compared to 2 years ago. 
I am one of the people who jumped into this field as well. I want to share with you guys my journey of switching into this field.

The decision

In 2017, I was a mechanical engineer who worked in the petrochemical factory in Rayong province, Thailand. 
Typically, I was making a presentation to the management team about why the machine failed or how to prevent the machine breakdown. 
At that time, I had a chance to utilize the data to ensure and answer most technical questions.

One day, I realized that Rayong is not a place for me to settle down. 
I wanted to go back to Bangkok, so I need to find a way to relocate myself. 
I was searching for a job, and it was hard to find a good one because my background is in mechanical engineering, and it is not that many good engineering jobs were placed in Bangkok

In [ ]:
import re

article_content = re.sub(r'\n|\r', ' ', article_content)
article_content = re.sub(r' +', ' ', article_content)
article_content = article_content.strip()

In [ ]:
sentences = nltk.sent_tokenize(article_content)
len(sentences)

75

In [ ]:
import numpy as np

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_sentences = normalize_corpus(sentences)
norm_sentences[:3]

array(['data science still attractive trend thailand nowadays',
       'many open positions data scientists data engineers datarelated roles compared years ago',
       'one people jumped field well'], dtype='<U163')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
dt_matrix = tv.fit_transform(norm_sentences)
dt_matrix = dt_matrix.toarray()

vocab = tv.get_feature_names_out()
td_matrix = dt_matrix.T
print(td_matrix.shape)
pd.DataFrame(np.round(td_matrix, 2), index=vocab).head(10)

(399, 75)


,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
ab,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
academic,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
accelerate,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
active,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
add,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
ago,0.0,0.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
ahead,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.37,0.0,0.0,0.0
also,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.23,0.0,0.0,0.0,0.00,0.0,0.0,0.0
analysis,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
analyst,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0


# Latent Semantic Analysis

Apply low-rank Singular Value Decomposition to this matrix. The core principle behind Latent Semantic Analysis (LSA) is that in any document, there exists a latent structure among terms that are related
contextually and hence should also be correlated in the same singular space.

The main idea in our implementation is to use SVD (recall M = USVT) so that U
and V are the orthogonal matrices and S is the diagonal matrix, which can also be
represented as a vector of the singular values. 

The original matrix can be represented as
a term-document matrix where the rows are terms and each column is a document, i.e.,
a sentence from our document in this case. The values can be any type of weighting like
Bag of Words model-based frequencies, TF-IDFs, or binary occurrences.

![](https://i.imgur.com/YtopNr3.png)

In [ ]:
from scipy.sparse.linalg import svds
    
def low_rank_svd(matrix, singular_count=2):
    u, s, vt = svds(matrix, k=singular_count)
    return u, s, vt

In [ ]:
num_sentences = 8
num_topics = 3

u, s, vt = low_rank_svd(td_matrix, singular_count=num_topics)  
print(u.shape, s.shape, vt.shape)
term_topic_mat, singular_values, topic_document_mat = u, s, vt

(399, 3) (3,) (3, 75)


In [ ]:
# remove singular values below threshold                                         
sv_threshold = 0.5
min_sigma_value = max(singular_values) * sv_threshold
singular_values[singular_values < min_sigma_value] = 0

In [ ]:
salience_scores = np.sqrt(np.dot(np.square(singular_values), 
                                 np.square(topic_document_mat)))
salience_scores

array([0.33492016, 0.52500692, 0.37348266, 0.47469627, 0.14054693,
       0.03439453, 0.23871801, 0.0787093 , 0.20686297, 0.23948814,
       0.44722253, 0.25374833, 0.17496657, 0.07736326, 0.16249226,
       0.12176427, 0.2852343 , 0.45772019, 0.29694581, 0.14007367,
       0.25993774, 0.04835826, 0.18155796, 0.33731336, 0.55012501,
       0.37296844, 0.09458874, 0.32731158, 0.01812493, 0.6605594 ,
       0.41860437, 0.2310217 , 0.39845551, 0.25243084, 0.09335506,
       0.35269007, 0.41065046, 0.31705163, 0.14529888, 0.19857846,
       0.52612016, 0.17893873, 0.09710242, 0.10334108, 0.09583107,
       0.42323164, 0.39859951, 0.27528109, 0.32500898, 0.30583542,
       0.31292114, 0.35921656, 0.30286234, 0.14607397, 0.29765589,
       0.0878497 , 0.06084374, 0.06686582, 0.17060446, 0.42001863,
       0.22537105, 0.22284131, 0.38907552, 0.55180705, 0.17343551,
       0.21005421, 0.30714613, 0.26765359, 0.10369973, 0.16297536,
       0.30081344, 0.34964022, 0.51733889, 0.26680969, 0.09080

In [ ]:
top_sentence_indices = (-salience_scores).argsort()[:num_sentences]
top_sentence_indices.sort()

In [ ]:
print('\n'.join(np.array(sentences)[top_sentence_indices]))

There are many open positions for data scientists, data engineers, and data-related roles compared to 2 years ago.
I want to share with you guys my journey of switching into this field.
Secondly, I entered the platform called “Kaggle.” That was a real starting point towards the data science field for me.
I would recommend anyone interested in this field to go for the Kaggle use cases or make it on your own.
In Thailand, there were not many open data scientist positions.
The benefit of being a data scientist is that there are many chances to touch many business operations areas.
A lot of people have delivered their analytics use cases to the in-house business.
I hope this encourages you guys who are interested in switching their job/responsibility towards this field.
